In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import pymysql.cursors
from sqlalchemy import create_engine
import json

# Sales data for ACME Corp.

## Project requirements

1. Which product offering is performing better 

2. Provide information on user demographics (age range, region) would help them hone in on marketing spend 

3. Are the sales seasonal or does it remain consistent throughout the year 

4. In the year 2021, our nationwide marketing spend was £200,000,000. Marketing activities covered the following regions - UK South East, UK North East, UK North, UK South. Can we justify the spend or was it a wasteful expenditure? 

5. We need to find out by region if our CPA – cost per acquisition is higher or lower than the average as per the research found on https://mystaticwebsite-3.s3.amazonaws.com/index.html 

In [3]:
data_url = "https://mystaticwebsite-3.s3.amazonaws.com/test-data.csv"
data = requests.get(data_url).content
file_path = "data/" + data_url.split('/')[-1]
with open(file_path, 'wb') as f:
    f.write(data)
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     50 non-null     float64
 1   product      50 non-null     object 
 2   quantity     50 non-null     float64
 3   order_month  50 non-null     object 
 4   age          50 non-null     float64
 5   uk_region    50 non-null     object 
 6   cpa          50 non-null     float64
dtypes: float64(4), object(3)
memory usage: 2.9+ KB


In [4]:
df[df.isna().any(axis=1)]

,order_id,product,quantity,order_month,age,uk_region,cpa
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# One row containing all nulls, no other nulls so can safely drop

df.dropna(inplace=True)

Assumptions for each column to check validation:
* order_id : must be unique positive int
* product : must be either "toys" or "toothbrush"
* quantity : must be positive int
* order_month : must be in the format mmm-yy
* age : must be positive int, should be a sensible number
* uk_region : must be either "UK South East", "UK North East", "UK North", or "UK South"
* cpa : must be positive numeric

In [6]:
for col in df.columns:
    print(df[col].describe())

count    50.00000
mean     25.50000
std      14.57738
min       1.00000
25%      13.25000
50%      25.50000
75%      37.75000
max      50.00000
Name: order_id, dtype: float64
count             50
unique             2
top       toothbrush
freq              35
Name: product, dtype: object
count    50.000000
mean      1.320000
std       0.586933
min       1.000000
25%       1.000000
50%       1.000000
75%       1.750000
max       3.000000
Name: quantity, dtype: float64
count         50
unique         5
top       Mar-21
freq          14
Name: order_month, dtype: object
count    50.000000
mean     30.840000
std      20.769247
min     -59.000000
25%      21.000000
50%      31.000000
75%      42.500000
max      60.000000
Name: age, dtype: float64
count                50
unique                3
top       UK South East
freq                 22
Name: uk_region, dtype: object
count    50.000000
mean      3.180000
std       1.955526
min       1.000000
25%       2.000000
50%       3.000000
75%      

In [7]:
print(f"Unique order ids: {df['order_id'].nunique()}")
print(f"Number of orders: {len(df)}")

Unique order ids: 50
Number of orders: 50


In [8]:
# order_id is fine, check product:
df['product'].value_counts()

toothbrush    35
toys          15
Name: product, dtype: int64

In [9]:
# product is fine, check quantity:
df[df['quantity'].map(lambda x: (x % 1 != 0) or (x < 0))]

,order_id,product,quantity,order_month,age,uk_region,cpa


In [10]:
# quantity is fine, check order month:
df[df["order_month"].map(lambda x: not re.match(r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)-[0-9]{2}", x))]

,order_id,product,quantity,order_month,age,uk_region,cpa


In [11]:
# order month is fine, check age:
df[df['age'].map(lambda x: (x % 1 != 0) or (x < 0) or (x > 100))]

,order_id,product,quantity,order_month,age,uk_region,cpa
21,22.0,toothbrush,1.0,Mar-21,-59.0,UK North East,5.0
31,32.0,toys,1.0,Jan-21,-31.0,UK South East,2.0


In [12]:
# looks like 2 ages have accidentally been entered with minus signs, this can be easily reversed:

df['age'] = abs(df['age'])

In [13]:
# check uk_region

df['uk_region'].value_counts()

UK South East    22
UK North East    20
UK North          8
Name: uk_region, dtype: int64

In [14]:
# no data for UK South, otherwise the column is fine.

Individual columns are cleaned successfully, however for analysis of seasonality it will be useful to separate out months from years. We will also need to set the types of relevant columns to int to make sure that unnecessary decimal places are not saved.

In [15]:
df['order_year'] = df['order_month'].map(lambda x: x[-2:])
df['order_month'] = df['order_month'].map(lambda x: x[:3])

for col in ['order_id', 'quantity', 'age', 'order_year']:
    df[col] = df[col].astype('int64')

# This is where I would put my database connector to do this programmatically - csv will be uploaded via MySQL Workbench instead

# df.to_csv('data/test-data-clean.csv', index=False)

In [20]:
with open("db_login.json", "r") as f:
    login = json.load(f)

conn = pymysql.connect(
    host=login['host'],
    user=login['user'],
    password=login['pass'],
    database=login['db_name'],
    cursorclass=pymysql.cursors.DictCursor,
    ssl={"fake_flag_to_enable_tls":True}
    # see link https://stackoverflow.com/questions/55617520/unable-to-make-tls-tcp-connection-to-remote-mysql-server-with-pymysql-other-too
)
engine = create_engine(f"mysql+pymysql://{login['user']}:{login['pass']}@{login['host']}/{login['db_name']}")

with conn:
    with conn.cursor() as cur:
        cur.execute("""
                    CREATE TABLE IF NOT EXISTS test_clean(
                    order_id INTEGER PRIMARY KEY NOT NULL,
                    product TEXT,
                    quantity INTEGER,
                    order_month VARCHAR(3),
                    age INTEGER,
                    uk_region VARCHAR(20),
                    cpa DOUBLE,
                    order_year INTEGER
        );""")
    conn.commit()
    df.to_sql('test_clean', engine, if_exists='append')

c:\Users\ChrisWilliams\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': not all arguments converted during string formatting